In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import re

In [2]:
# Thử nghiệm url đầu tiên
url = "https://tuoitre.vn/timeline/0/trang-1.htm"

# Lấy danh sách news_items
html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.findAll('li', class_='news-item')

# Lấy ra link, title và category từ news_items
raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
for item in news_items:
    title = item.find('h3', class_='title-news').text.replace('\n','')
    link = "https://tuoitre.vn" + item.find('a').attrs["href"]
    category = item.find('a', class_='category-name').text
    raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)

raw_data.head()

,links,title,description,content,class
0,https://tuoitre.vn/thu-mon-ha-lan-mo-hang-nam-...,Thủ môn Hà Lan mở hàng năm mới 'xúi quẩy' với ...,NaN,NaN,Thể thao vui
1,https://tuoitre.vn/video-vo-mua-tra-sua-roi-tr...,"Video: Vờ mua trà sữa rồi trộm iPhone, bị ngườ...",NaN,NaN,Pháp luật
2,https://tuoitre.vn/noi-da-ga-khi-xem-vdv-truot...,'Nổi da gà' khi xem VĐV trượt tuyết lộn vòng '...,NaN,NaN,Thể thao
3,https://tuoitre.vn/lich-su-eo-le-ong-biden-tun...,"Lịch sử éo le, ông Biden từng chủ trì tuyên bố...",NaN,NaN,Thế giới
4,https://tuoitre.vn/nam-2021-tang-truong-tin-du...,Năm 2021 tăng trưởng tín dụng được dự báo ra sao?,NaN,NaN,Kinh doanh


In [3]:
for _, row in raw_data.iterrows():
    news_page = requests.get(row["links"]).content
    news_tree = BeautifulSoup(news_page, "html.parser")
    # Lấy mô tả
    try:
        row["description"] = news_tree.find("h2", class_="sapo").text
    except:
        row["description"] = ''
    # Lấy nội dung
    try:
        body = news_tree.find("div", id="main-detail-body")
        content = body.findChildren("p", recursive=False)
        row["content"] = ""
        for x in content:
            row["content"] += x.text
    except:
        row["content"] = ''
    time.sleep(0.05)
    
raw_data.head()

,links,title,description,content,class
0,https://tuoitre.vn/thu-mon-ha-lan-mo-hang-nam-...,Thủ môn Hà Lan mở hàng năm mới 'xúi quẩy' với ...,TTO - Thủ môn 26 tuổi người Hà Lan Norbert Alb...,Trận đấu trong khuôn khổ vòng 18 Giải hạng nhấ...,Thể thao vui
1,https://tuoitre.vn/video-vo-mua-tra-sua-roi-tr...,"Video: Vờ mua trà sữa rồi trộm iPhone, bị ngườ...",TTO - Thanh niên tên Trần Minh Trí đến cửa tiệ...,"Ngày 6-1, nguồn tin của Tuổi Trẻ Online xác nh...",Pháp luật
2,https://tuoitre.vn/noi-da-ga-khi-xem-vdv-truot...,'Nổi da gà' khi xem VĐV trượt tuyết lộn vòng '...,TTO - VĐV trượt tuyết tự do người Thụy Sĩ Fabi...,Cú nhảy siêu phàm trên đã được Fabian Bösch th...,Thể thao
3,https://tuoitre.vn/lich-su-eo-le-ong-biden-tun...,"Lịch sử éo le, ông Biden từng chủ trì tuyên bố...","TTO - Trên lý thuyết, đây là cơ hội chính thức...","Cho đến giờ chót, cơ sở để ông Donald Trump đò...",Thế giới
4,https://tuoitre.vn/nam-2021-tang-truong-tin-du...,Năm 2021 tăng trưởng tín dụng được dự báo ra sao?,TTO - Tăng trưởng tín dụng năm 2021 được dự bá...,Công ty chứng khoán SSI đã đưa ra dự báo này t...,Kinh doanh


### Vấn đề phát sinh trong quá trình thu thập dữ liệu tự động hàng loạt

### Thiết lập đoạn code thu thập dữ liệu tự động hàng loạt

In [3]:
def single_request_scraping(index, sleep_time = 0.05, limit_retry = 100):
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('li', class_='news-item')
    
    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    # Lấy ra link, title và category từ news_items
    raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
    for item in news_items:
        try:
            title = item.find('h3', class_='title-news').text.replace('\n','')
        except:
            title = ""
        link = "https://tuoitre.vn" + item.find('a').attrs["href"]
        try:
            category = item.find('a', class_='category-name').text
        except:
            category = ""
        raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)
    
    # Tiến hành lấy nội dung từng link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            row["description"] = ""
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            row["description"] = news_tree.find("h2", class_="sapo").text
        except:
            row["description"] = ''
        # Lấy nội dung
        try:
            body = news_tree.find("div", id="main-detail-body")
            content = body.findChildren("p", recursive=False)
            row["content"] = ""
            for x in content:
                row["content"] += x.text
        except:
            row["content"] = ''
        time.sleep(sleep_time)
    
    return raw_data

In [3]:
#Batch scraping
iter_num = 40
num = 200
continue_flag = True

while (continue_flag):
    batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
    for index in range(iter_num*num+1,(iter_num+1)*num+1):
        data = single_request_scraping(index,0)
        if (data is None):
            continue_flag = False
            break
        print(f"Page {index} complete!")
        batch_df = batch_df.append(data)
    batch_df.to_csv(f'scraped_data\crawling_{iter_num}.csv',index=False,encoding="utf-8")    
    if (not continue_flag): break
    else: iter_num+=1

Page 8001 complete!
Page 8002 complete!
Page 8003 complete!
Page 8004 complete!
Page 8005 complete!
Page 8006 complete!
Page 8007 complete!
Page 8008 complete!
Page 8009 complete!
Page 8010 complete!
Page 8011 complete!
Page 8012 complete!
Loi 502 tai link https://tuoitre.vn/tphcm-tro-giup-kip-thoi-doi-tuong-gap-kho-khan-dot-xuat-1351942.htm
Page 8013 complete!
Page 8014 complete!
Page 8015 complete!
Page 8016 complete!
Page 8017 complete!
Page 8018 complete!
Page 8019 complete!
Page 8020 complete!
Page 8021 complete!
Page 8022 complete!
Page 8023 complete!
Page 8024 complete!
Page 8025 complete!
Page 8026 complete!
Page 8027 complete!
Page 8028 complete!
Page 8029 complete!
Page 8030 complete!
Page 8031 complete!
Page 8032 complete!
Page 8033 complete!
Page 8034 complete!
Page 8035 complete!
Page 8036 complete!
Page 8037 complete!
Page 8038 complete!
Page 8039 complete!
Page 8040 complete!
Page 8041 complete!
Page 8042 complete!
Page 8043 complete!
Page 8044 complete!
Page 8045 comple

Page 8270 complete!
Page 8271 complete!
Page 8272 complete!
Page 8273 complete!
Page 8274 complete!
Page 8275 complete!
Page 8276 complete!
Page 8277 complete!
Page 8278 complete!
Loi 502 tai link https://tuoitre.vn/tu-nguoi-rua-chen-thue-thanh-chu-chuoi-nha-hang-chuc-trieu-do-1325625.htm
Page 8279 complete!
Page 8280 complete!
Page 8281 complete!
Page 8282 complete!
Page 8283 complete!
Page 8284 complete!
Page 8285 complete!
Page 8286 complete!
Page 8287 complete!
Page 8288 complete!
Page 8289 complete!
Page 8290 complete!
Page 8291 complete!
Page 8292 complete!
Page 8293 complete!
Page 8294 complete!
Page 8295 complete!
Page 8296 complete!
Page 8297 complete!
Page 8298 complete!
Page 8299 complete!
Page 8300 complete!
Page 8301 complete!
Page 8302 complete!
Page 8303 complete!
Page 8304 complete!
Page 8305 complete!
Page 8306 complete!
Page 8307 complete!
Page 8308 complete!
Page 8309 complete!
Page 8310 complete!
Page 8311 complete!
Page 8312 complete!
Page 8313 complete!
Page 8314 

Page 8608 complete!
Page 8609 complete!
Page 8610 complete!
Page 8611 complete!
Page 8612 complete!
Page 8613 complete!
Page 8614 complete!
Loi 502 tai link https://tuoitre.vn/giao-dich-bat-dong-san-soi-dong-o-phan-khuc-trung-va-cao-cap-1296635.htm
Page 8615 complete!
Page 8616 complete!
Page 8617 complete!
Loi request tai link https://tuoitre.vn/nghe-chua-benh-cho-do-thi-1295464.htm
Loi request tai link https://tuoitre.vn/bi-quyet-san-hoc-bong-tai-cac-truong-hang-dau-hoa-ky-1296212.htm
Page 8618 complete!
Page 8619 complete!
Page 8620 complete!
Page 8621 complete!
Page 8622 complete!
Page 8623 complete!
Page 8624 complete!
Page 8625 complete!
Page 8626 complete!
Page 8627 complete!
Page 8628 complete!
Page 8629 complete!
Loi request tai link https://tuoitre.vn/nganh-khi-tuong-thuy-van-kho-nhung-day-hap-dan-1295319.htm
Page 8630 complete!
Page 8631 complete!
Page 8632 complete!
Page 8633 complete!
Page 8634 complete!
Page 8635 complete!
Page 8636 complete!
Page 8637 complete!
Loi 502 t

Page 8913 complete!
Page 8914 complete!
Page 8915 complete!
Page 8916 complete!
Page 8917 complete!
Page 8918 complete!
Page 8919 complete!
Page 8920 complete!
Page 8921 complete!
Page 8922 complete!
Page 8923 complete!
Page 8924 complete!
Page 8925 complete!
Page 8926 complete!
Page 8927 complete!
Page 8928 complete!
Page 8929 complete!
Page 8930 complete!
Page 8931 complete!
Page 8932 complete!
Page 8933 complete!
Page 8934 complete!
Page 8935 complete!
Page 8936 complete!
Page 8937 complete!
Page 8938 complete!
Page 8939 complete!
Page 8940 complete!
Page 8941 complete!
Page 8942 complete!
Page 8943 complete!
Page 8944 complete!
Page 8945 complete!
Page 8946 complete!
Page 8947 complete!
Page 8948 complete!
Page 8949 complete!
Page 8950 complete!
Page 8951 complete!
Page 8952 complete!
Page 8953 complete!
Page 8954 complete!
Page 8955 complete!
Page 8956 complete!
Page 8957 complete!
Page 8958 complete!
Page 8959 complete!
Page 8960 complete!
Page 8961 complete!
Page 8962 complete!


Page 9284 complete!
Page 9285 complete!
Page 9286 complete!
Page 9287 complete!
Page 9288 complete!
Page 9289 complete!
Page 9290 complete!
Page 9291 complete!
Page 9292 complete!
Page 9293 complete!
Page 9294 complete!
Page 9295 complete!
Page 9296 complete!
Page 9297 complete!
Page 9298 complete!
Page 9299 complete!
Page 9300 complete!
Page 9301 complete!
Page 9302 complete!
Page 9303 complete!
Page 9304 complete!
Page 9305 complete!
Page 9306 complete!
Page 9307 complete!
Page 9308 complete!
Page 9309 complete!
Page 9310 complete!
Page 9311 complete!
Page 9312 complete!
Page 9313 complete!
Page 9314 complete!
Page 9315 complete!
Page 9316 complete!
Page 9317 complete!
Page 9318 complete!
Page 9319 complete!
Page 9320 complete!
Page 9321 complete!
Page 9322 complete!
Page 9323 complete!
Page 9324 complete!
Page 9325 complete!
Page 9326 complete!
Page 9327 complete!
Page 9328 complete!
Page 9329 complete!
Page 9330 complete!
Page 9331 complete!
Page 9332 complete!
Page 9333 complete!


...

Page 43014 complete!
Page 43015 complete!
Page 43016 complete!
Page 43017 complete!
Page 43018 complete!
Page 43019 complete!
Page 43020 complete!
Page 43021 complete!
Page 43022 complete!
Page 43023 complete!
Page 43024 complete!
Page 43025 complete!
Page 43026 complete!
Page 43027 complete!
Page 43028 complete!
Page 43029 complete!
Page 43030 complete!
Page 43031 complete!
Page 43032 complete!
Page 43033 complete!
Page 43034 complete!
Page 43035 complete!
Page 43036 complete!
Page 43037 complete!
Page 43038 complete!
Page 43039 complete!
Page 43040 complete!
Page 43041 complete!
Page 43042 complete!
Page 43043 complete!
Page 43044 complete!
Page 43045 complete!
Page 43046 complete!
Page 43047 complete!
Page 43048 complete!
Page 43049 complete!
Page 43050 complete!
Page 43051 complete!
Page 43052 complete!
Page 43053 complete!
Page 43054 complete!
Page 43055 complete!
Page 43056 complete!
Page 43057 complete!
Page 43058 complete!
Page 43059 complete!
Page 43060 complete!
Page 43061 co

Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link https://tuoitre.vn/al-qaeda-‘nhan-trach-nhiem-2-vu-no-bom-o-tho-nhi-ky’-9178.htm
Loi 404 tai link htt

Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/worldpay-bi-tin-tac-‘ha-guc’-7920.htm
Loi 404 tai link https://tuoitre.vn/wo

Page 43363 complete!
Page 43364 complete!
Page 43365 complete!
Page 43366 complete!
Page 43367 complete!
Page 43368 complete!
Page 43369 complete!
Page 43370 complete!
Page 43371 complete!
Page 43372 complete!
Page 43373 complete!
Page 43374 complete!
Page 43375 complete!
Page 43376 complete!
Page 43377 complete!
Page 43378 complete!
Page 43379 complete!
Page 43380 complete!
Page 43381 complete!
Page 43382 complete!
Page 43383 complete!
Page 43384 complete!
Page 43385 complete!
Page 43386 complete!
Page 43387 complete!
Page 43388 complete!
Page 43389 complete!
Page 43390 complete!
Page 43391 complete!
Page 43392 complete!
Page 43393 complete!
Page 43394 complete!
Page 43395 complete!
Page 43396 complete!
Page 43397 complete!
Page 43398 complete!
Page 43399 complete!
Page 43400 complete!
Page 43401 complete!
Page 43402 complete!
Page 43403 complete!
Page 43404 complete!
Page 43405 complete!
Page 43406 complete!
Page 43407 complete!
Page 43408 complete!
Page 43409 complete!
Page 43410 co

Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm
Loi 404 tai link https://tuoitre.vn/virus-swen-‘do-bo’-vao-viet-nam-2273.htm

In [4]:
# Thu thập các bài báo mới nhất
iter_num = 0
num = 400

batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
for index in range(iter_num*num+1,(iter_num+1)*num+1):
    data = single_request_scraping(index,0)
    if (data is None):
        continue_flag = False
        break
    print(f"Page {index} complete!")
    batch_df = batch_df.append(data)
batch_df.to_csv(f'scraped_data\crawling_{iter_num}.csv',index=False,encoding="utf-8")    

Page 1 complete!
Page 2 complete!
Page 3 complete!
Page 4 complete!
Page 5 complete!
Page 6 complete!
Page 7 complete!
Page 8 complete!
Page 9 complete!
Page 10 complete!
Page 11 complete!
Page 12 complete!
Page 13 complete!
Page 14 complete!
Page 15 complete!
Page 16 complete!
Page 17 complete!
Page 18 complete!
Page 19 complete!
Page 20 complete!
Page 21 complete!
Page 22 complete!
Page 23 complete!
Page 24 complete!
Page 25 complete!
Page 26 complete!
Page 27 complete!
Page 28 complete!
Page 29 complete!
Page 30 complete!
Page 31 complete!
Page 32 complete!
Page 33 complete!
Page 34 complete!
Page 35 complete!
Page 36 complete!
Page 37 complete!
Page 38 complete!
Page 39 complete!
Page 40 complete!
Page 41 complete!
Page 42 complete!
Page 43 complete!
Page 44 complete!
Page 45 complete!
Page 46 complete!
Page 47 complete!
Page 48 complete!
Page 49 complete!
Page 50 complete!
Page 51 complete!
Page 52 complete!
Page 53 complete!
Page 54 complete!
Page 55 complete!
Page 56 complete!
P